In [0]:
from openai import OpenAI
import os

import os
from openai import AzureOpenAI
    

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [0]:
def set_open_params(
    model="gpt-35-turbo",
    temperature=0.9,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(client, params, messages):
    """ GET completion from openai api"""

    response = client.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

In [0]:
from IPython.display import Markdown

# Zero-Shot prompt example

In [0]:
prompt = "write a pyspark code to read and display StoreNumber, ItemNumber,BusinessDate, TotalSalesAmount, TotalDiscountAmount and RevenueItemInd from the delta table pos_transactions. Add a column called NDSAmount which will have value of 0 if RevenueItemInd is Y else TotalSalesAmount - TotalDiscountAmount. Also count and display how many revenue items are sold in the store for each day"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(client, params, messages)
Markdown(response.choices[0].message.content)

```python
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# create a Spark session
spark = SparkSession.builder.appName("POS Transactions").getOrCreate()

# read the delta table pos_transactions
pos_transactions_df = spark.read.format("delta").load("/path/to/pos_transactions")

# select and display required columns
selected_df = pos_transactions_df.select("StoreNumber", "ItemNumber", "BusinessDate", "TotalSalesAmount", "TotalDiscountAmount", "RevenueItemInd")

# add NDSAmount column
selected_df = selected_df.withColumn("NDSAmount", F.when(F.col("RevenueItemInd") == "Y", 0).otherwise(F.col("TotalSalesAmount") - F.col("TotalDiscountAmount")))

# count revenue items sold in the store for each day
revenue_items_count = selected_df.filter(F.col("RevenueItemInd") == "Y").groupby("StoreNumber", "BusinessDate").agg(F.count("ItemNumber").alias("RevenueItemsSold"))

# display the dataframe
selected_df.show()
revenue_items_count.show()

# stop the Spark session
spark.stop()
```

# Few-Shot prompt example

In [0]:
prompt = """write a pyspark code to read and display StoreNumber, ItemNumber,BusinessDate, TotalSalesAmount, TotalDiscountAmount and RevenueItemInd from the delta table pos_transactions. 
Add a column called NDSAmount which will have value of 0 if RevenueItemInd is Y else TotalSalesAmount - TotalDiscountAmount. Use case when statement to find NDSAmount.
Also count and display how many revenue items are sold in the store for each businessdate. 
For example, if 5 items are marked as revenueItemInd as Y then the sum of remaining items when revenueItemInd is N gives this value"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(client, params, messages)
Markdown(response.choices[0].message.content)

```python
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create a Spark session
spark = SparkSession.builder.appName("pos_transactions_analysis").getOrCreate()

# Read the delta table pos_transactions
pos_transactions = spark.read.format("delta").load("path_to_delta_table")

# Display StoreNumber, ItemNumber, BusinessDate, TotalSalesAmount, TotalDiscountAmount, RevenueItemInd
# and calculate NDSAmount based on RevenueItemInd
pos_transactions = pos_transactions.withColumn("NDSAmount",
                                               F.when(F.col("RevenueItemInd") == "Y", 0)
                                               .otherwise(F.col("TotalSalesAmount") - F.col("TotalDiscountAmount")))
pos_transactions.select("StoreNumber", "ItemNumber", "BusinessDate", "TotalSalesAmount",
                       "TotalDiscountAmount", "RevenueItemInd", "NDSAmount").show()

# Count and display how many revenue items are sold in the store for each businessdate
revenue_items_count = pos_transactions.filter(pos_transactions["RevenueItemInd"] == "Y") \
    .groupBy("StoreNumber", "BusinessDate").agg(F.count("*").alias("RevenueItemsCount"))
revenue_items_count.show()
```